# Aufbereitung Abstimmungsdaten Grossrat
Im Berner Grossrat wird elektronisch abgestimmt. Allerdings wechselte die Anlage und damit das Datenformat in der laufenden Legislatur mehrfach. So kam bei der ersten Session, die wegen Corona im Messeareal BEA Expo durchgeführt wurde, eine andere Anlage zum Einsatz als zuvor im Rathaus. Und bei der zweiten Expo-Session entstand wiederum ein anders gegliederter Datensatz. In den Daten, die von [Roland Schneeberger](roland.schneeberger@be.ch) von der Staatskanzlei exportiert worden sind, sind teilweise zudem Testabstimmungen enthalten. Diese müssen entfernt werden. Weitere Herausforderung: Unter- und Variantenabstimmungen haben das selbe Gewicht wie Schlussabstimmungen. 

In diesem Notebook werden die Daten der verschiedenen Abstimmungsanlagen aufbereitet und harmonisiert. 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Rathaus" data-toc-modified-id="Rathaus-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Rathaus</a></span></li><li><span><a href="#BEA-Session-1" data-toc-modified-id="BEA-Session-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>BEA-Session 1</a></span></li><li><span><a href="#BEA-Session-2" data-toc-modified-id="BEA-Session-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BEA-Session 2</a></span></li><li><span><a href="#Harmonisieren-und-zusammenfügen-der-Daten" data-toc-modified-id="Harmonisieren-und-zusammenfügen-der-Daten-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Harmonisieren und zusammenfügen der Daten</a></span></li></ul></div>

In [1]:
# Die nötigen Bilbiotheken werden importiert. 
import pandas as pd
from os import listdir
from tqdm import tqdm
import numpy as np
from fuzzywuzzy import process

In [2]:
# Welche Datei zu welcher Session gehört, wird manuell erfasst. 
format_rathaus = listdir('daten/rathaus/')
format_bea_v1 = listdir('daten/bea1/')
format_bea_v2 = listdir('daten/bea2/')

# Sperrdateien, die beim Öffnen eines Datensatzes in Office-Programmen entstehen, werden ausgeschlossen.
format_rathaus = [x for x in format_rathaus if 'lock' not in x]
format_bea_v1 = [x for x in format_bea_v1 if 'lock' not in x]
format_bea_v2 = [x for x in format_bea_v2 if 'lock' not in x]

In [3]:
# Eine Liste mit den Parlamentariern wird eingelesen.  
df_grossrat = pd.read_excel('https://www.gr.be.ch/gr/de/index/mitglieder/mitglieder/liste_a-z.assetref/dam/documents/GR/Mitglieder/de/Adressliste--Liste-des-adresses.xlsx')
df_grossrat = df_grossrat.dropna(how='all')
df_grossrat['name-vorname'] = df_grossrat['Name'] + ' ' + df_grossrat['Vornamen']
df_grossrat['ort'] = df_grossrat['PLZOrt'].apply(lambda x: x.split(' ')[1].strip() if len(x.split(' ')) > 1 else x)

ehemalige = [['Luca', 'Alberucci', 'glp', 'Ostermundigen', 'd'], 
             ['Christian', 'Bachmann', 'SP', 'Nidau', 'd'],
             ['Peter', 'Sommer', 'FDP', 'Wynigen', 'd'],
             ['Madeleine', 'Graf-Rudolf', 'Grüne', 'Belp', 'd'],
             ['Marianne', 'Teuscher-Abts', 'FDP', 'Roggwil', 'd'],
             ['Marianne', 'Burkhard', 'SP', 'Roggwil', 'd'],
             ['Jakob', 'Etter', 'BDP', 'Treiten', 'd'],
             ['Christine', 'Schnegg', 'EVP', 'Lyss', 'd'],
             ['Peter', 'Moser', 'FDP', 'Biel/Bienne', 'd'],
             ['Stefan', 'Hofer', 'SVP', 'Bern', 'd'],
             ['Roland', 'Näf', 'SP', 'Muri bei Bern', 'd'],
             ['Ulrich', 'Stähli', 'BDP', 'Gasel', 'd'],
             ['Ulrich', 'Frutiger', 'BDP', 'Oberhofen', 'd'],
             ['Erich', 'Hess', 'SVP', 'Bern', 'd'],
             ['Vania', 'Kohli', 'BDP', 'Bern', 'd'],
             ['Samuel', 'Leuenberger', 'BDP', 'Trubschachen', 'd'],
             ['Martin', 'Aeschlimann', 'EVP', 'Burgdorf', 'd'],
             ['Monika', 'Gygax-Böninger', 'BDP', 'Obersteckholz', 'd'],
             ['Ruedi', 'Löffel', 'EVP', 'Münchenbuchsee', 'd'],
             ['Marc', 'Jost', 'EVP', 'Thun', 'd'],
             ['Jürg', 'Iseli', 'SVP', 'Zwieselberg', 'd'],
             ['Anita', 'Luginbühl-Bachmann', 'BDP', 'Krattigen', 'd'],
             ['Lars', 'Guggisberg', 'SVP', 'Kirchlindach', 'd'],
             ['Lea', 'Kusano', 'SP', 'Bern', 'd'],
             ['Tamara', 'Funiciello', 'SP', 'Bern', 'd'], 
             ['Kilian', 'Baumann', 'Grüne', 'Suberg', 'd'],
             ['Daniel', 'Trüssel', 'Grünliberale', 'Trimstein', 'd']]

df_ehemalige = pd.DataFrame(ehemalige)
df_ehemalige.columns = ['vorname', 'nachname', 'partei', 'wohnort', 'sprache']
df_ehemalige = df_ehemalige[['nachname', 'vorname', 'partei', 'wohnort', 'sprache']]

df_grossrat = df_grossrat[['Name', 'Vornamen', 'Partei', 'ort', 'Sprache']]
df_grossrat.columns = ['nachname', 'vorname', 'partei', 'wohnort', 'sprache']

df_grossrat = df_grossrat.append(df_ehemalige)
df_grossrat.head()

,nachname,vorname,partei,wohnort,sprache
0,Abplanalp,Ueli,SVP,Brienzwiler,d
1,Aebi,Markus,SVP,Hellsau,d
2,Aebischer,Verena,SVP,Guggisberg,d
3,Alberucci,Luca,glp,Ostermundigen,d
4,Ammann,Christa,AL,Bern,d


In [4]:
# Die Namen der Parteien sind unterschiedlich erfasst. Sie werden harmonisiert. 
parteien_dict = {'SVP / UDC': 'SVP', 'SP-JUSO-PSA / PS-JS-PSA': 'SP', 'FDP / PLR': 'FDP',
                 'Grüne / Les Verts': 'Grüne', 'BDP / PBD': 'Die Mitte', 'glp / pvl': 'glp', 
                 'EVP / PEV': 'EVP', 'EDU / UDF': 'EDU', 'CVP / PDC': 'CVP', 'BDP': 'Die Mitte',
                 'PLR': 'FDP', 'PSA': 'SP', 'CVP': 'Die Mitte', 'PS': 'SP', 'les Verts': 'Grüne',
                 'UDC': 'SVP', 'PEV': 'EVP', 'PDC': 'Die Mitte', 'SVP/UDC': 'SVP', 'FDP/PLR': 'FDP',
                 'Grüne/les Verts': 'Grüne', 'glp/plv': 'glp', 'EVP/PEV': 'EVP', 'EDU/UDF': 'EDU',
                 'Grünliberale': 'glp', 'Les Verts': 'Grüne'}

for i, r in df_grossrat.iterrows():
    if r['partei'] in parteien_dict.keys():
        df_grossrat.at[i, 'partei'] = parteien_dict[r['partei']]

## Rathaus
Bis im Frühling 2020 fanden die Sessionen im Rathaus statt. Die fix installierte Abstimmungsanlage produziert CSV. In den Daten sind allerdings viele Testabstimmungen enthalten. Roland Schneeberger hat diese ausgefiltert. Und er hat die Daten ins Excel-Format konvertiert. Diese Daten werden nachfolgend eingelesen. 

In [5]:
# Ein leerer Dataframe wird erstellt. 
cols = ['nachname', 'wohnort', 'partei', 'stimme']
df_rathaus = pd.DataFrame(columns=cols)

for dateiname in tqdm(format_rathaus):

    # Die Namen der Tabs der betreffenden Datei werden ausgelesen. 
    register = pd.ExcelFile('daten/rathaus/' + dateiname).sheet_names

    for r in register:
        # Die Abstimmungsdaten werden eingelesen. 
        df_roh = pd.read_excel('daten/rathaus/' + dateiname, sheet_name=r, skiprows=1)
        df_roh.columns = cols

        # Die Metainformationen zu den Abstimmungen werden ausgelesen (Datum, Resultat sowie Position in der Datei).
        anfang = df_roh[df_roh['nachname'] == 'Name/Nom'].index
        ende = df_roh[df_roh['nachname'] == '####################'].index

        temp_list = list()
        for i in range(len(ende)):
            if i % 2 == 0:
                temp_list.append(ende[i])
        ende = temp_list

        datetime = df_roh[df_roh['nachname'] == 'Datum/Date']['wohnort'].tolist()
        ja = df_roh[df_roh['nachname'] == 'Ja/Oui']['wohnort'].tolist()
        nein = df_roh[df_roh['nachname'] == 'Nein/Non']['wohnort'].tolist()
        enthalten = df_roh[df_roh['nachname'] == 'Enthalten/Abstentions']['wohnort'].tolist()
        art = df_roh[df_roh['nachname'] == 'Art/Sorte']['wohnort'].tolist()
        mandant = df_roh[df_roh['nachname'] == 'Mandant/Mandant']['wohnort'].tolist()
        nummer = df_roh[df_roh['nachname'] == 'Nummer/Numéro']['wohnort'].tolist()
        traktandum = df_roh[df_roh['nachname'] == 'Traktandum/Affair']['wohnort'].tolist()
        traktandum_txt = df_roh[df_roh['nachname'] == 'Traktandum Beschreibung/Affair description']['wohnort'].tolist()

        tab_meta = list(zip(anfang, ende, datetime, ja, nein, enthalten, art, mandant, nummer, traktandum, traktandum_txt))

        # Der Datensatz wird zusammengestellt. 
        for i in range(len(tab_meta)): 
            df_temp = df_roh.loc[tab_meta[i][0] + 1: tab_meta[i][1] - 1].copy()
            if type(tab_meta[i][2]) == float:
                df_temp['datetime'] = pd.to_datetime(r, dayfirst=True)
            else:
                df_temp['datetime'] = tab_meta[i][2]
            df_temp['res_ja'] = tab_meta[i][3] 
            df_temp['res_nein'] = tab_meta[i][4]   
            df_temp['res_enthaltung'] = tab_meta[i][5]
            df_temp['art'] = tab_meta[i][6]
            df_temp['mandant'] = tab_meta[i][7]
            df_temp['nummer'] = tab_meta[i][8]
            
            # Einzelne Abstimmungen tragen die selbe Nummer. Sie werden unterscheidbar gemacht. 
            df_temp['traktandum'] = tab_meta[i][9] + '_' + str(i)

            df_temp['traktandum_txt'] = tab_meta[i][10]
            df_temp['dateiname'] = dateiname
            df_temp['register'] = r
            df_rathaus = df_rathaus.append(df_temp)

# Stimmen, die unter dem Namen "Delegates" abgegeben wurden, werden ausgefiltert. 
df_rathaus = df_rathaus[~df_rathaus['nachname'].str.contains('Delegate')]

df_rathaus['anlage'] = 'rathaus'
df_rathaus.to_csv('daten/daten_rathaus.csv')
df_rathaus.head()

100%|██████████| 8/8 [01:15<00:00,  9.42s/it]


,nachname,wohnort,partei,stimme,datetime,res_ja,res_nein,res_enthaltung,art,mandant,nummer,traktandum,traktandum_txt,dateiname,register,anlage
13,Iseli,Zwieselberg,SVP / UDC,NaN,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
14,Aebi,Hellsau,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
15,Aebischer,Riffenmatt,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
16,Amstutz,Schwanden-Sigriswil,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
17,Augstburger,Gerzensee,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus


In [6]:
# Im Datenatz fehlen die Vornamen. Diese werden eingefügt. 
df_temp = df_rathaus[['nachname', 'wohnort', 'partei']].drop_duplicates()
df_temp = pd.merge(df_temp, df_grossrat, left_on=['nachname', 'wohnort'], right_on=['nachname', 'wohnort'], how='left')
df_temp = df_temp[['nachname', 'vorname', 'wohnort', 'partei_x']]
df_temp.columns = ['nachname', 'vorname', 'wohnort', 'partei']
df_temp['test'] = df_temp['nachname'] + df_temp['wohnort'] + df_temp['partei']

# Leider wurden -- z.B. wegen unterschiedlichen Ortsangaben in den beiden Datensätzen -- nicht alle
# Vornamen gefunden. Deshalb wird nun ein ungenaues Matching durchgeführt. 
df_grossrat['test'] = df_grossrat[['nachname', 'wohnort', 'partei']].sum(axis=1).tolist()

#kontrollliste = df_temp[df_temp['vorname'].isnull()].index

#testliste = df_grossrat[['nachname', 'wohnort', 'partei']].sum(axis=1).tolist()
for i, r in df_temp[df_temp['vorname'].isnull()].iterrows():
    best = process.extract(r['test'], df_grossrat['test'].tolist())
    df_temp.at[i, 'vorname'] = df_grossrat[df_grossrat['test'] == best[0][0]]['vorname'].values[0]
    
del df_grossrat['test']
del df_temp['test']

# Die Tabelle mit den vollen Namen wird dem Originaldatensatz hinzugefügt. 
df_temp = pd.merge(df_rathaus, df_temp[['nachname', 'vorname', 'partei']], left_on=['nachname', 'partei'], right_on=['nachname', 'partei'], how='left')
df_temp = df_temp.drop_duplicates(subset=['nachname', 'wohnort', 'stimme', 'traktandum', 'dateiname', 'register'])

In [7]:
# Fehlende Spalten werden eingefügt. Datensatz wird richtig gegliedert. 
df_temp['res_keine_stimme'] = np.NaN
df_temp = df_temp[['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'traktandum', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']]
df_temp.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
df_temp = df_temp.drop_duplicates()
df_rathaus = df_temp.copy()

## BEA-Session 1

In [ ]:
# Die Excel-Dateien (Version 1) werden eingelesen. 
df_bea_1 = pd.DataFrame()

for dateiname in tqdm(format_bea_v1):

    # Durch die Register loopen...
    register = pd.ExcelFile('daten/bea1/' + dateiname).sheet_names

    for r in register:
    
        # Die Metadaten werden ausgelesen. 
        df_temp = pd.read_excel('daten/bea1/' + dateiname, sheet_name=r)
        temp_geschäftsname = ' '.join([x for x in pd.read_excel('daten/bea1/' + dateiname, sheet_name=r).columns if 'Unnamed' not in x])
        temp_datetime = df_temp[df_temp.iloc[:,0] == 'Date and Time:'].iloc[0,1]
        temp_ja = df_temp[df_temp.iloc[:,0] == 'Yes votes:'].iloc[0,1]
        temp_nein = df_temp[df_temp.iloc[:,0] == 'No votes:'].iloc[0,1]
        temp_enthaltungen = df_temp[df_temp.iloc[:,0] == 'Abstentions:'].iloc[0,1]
        temp_keine_stimme = df_temp[df_temp.iloc[:,0] == 'Not voting:'].iloc[0,1]

        # Der Abstimmungsresultate werden eingelesen. 
        df_temp_roh = pd.read_excel('daten/bea1/' + dateiname, sheet_name=r, skiprows=12)
        df_temp_roh = df_temp_roh.rename(columns={'Anrede': 'TED'})
        
        # Die Ja, Nein, Enthaltungen sind in unter einander stehenden Tabellen aufgeführt.
        # Deshalb muss mit Subsets gearbeitet werden. 
        schnittmarke_nein = df_temp_roh[df_temp_roh['TED'] == 'Against'].iloc[0].name
        schnittmarke_enthaltung = df_temp_roh[df_temp_roh['TED'] == 'Abstentions'].iloc[0].name

        df_temp = pd.DataFrame()
        for v in ['ja', 'nein', 'enthaltung']:
            if v == 'ja':
                df_temp1 = df_temp_roh.copy().iloc[:schnittmarke_nein]
                df_temp1['stimme'] = 'Ja / Oui'
            elif v == 'nein':
                df_temp1 = df_temp_roh.copy().iloc[schnittmarke_nein + 1:schnittmarke_enthaltung]
                df_temp1['stimme'] = 'Nein / Non'
            elif v == 'enthaltung':
                df_temp1 = df_temp_roh.copy().iloc[schnittmarke_enthaltung + 1:]
                df_temp1['stimme'] = 'Enthaltungen / Abstentions'
            df_temp = df_temp.append(df_temp1)

        # Die Metadaten werden hinzugefügt.
        df_temp['geschäftsname'] = temp_geschäftsname
        df_temp['datetime'] = temp_datetime
        df_temp['res_ja'] = temp_ja
        df_temp['res_nein'] = temp_nein
        df_temp['enthaltungen'] = temp_enthaltungen
        df_temp['keine_stimme'] = temp_keine_stimme
        df_temp['dateiname'] = dateiname
        df_temp['register'] = r

        # Die neuen Daten werden dem Datensatz angehängt. 
        df_bea_1 = df_bea_1.append(df_temp)  

# Die Daten werden bereinigt. 
df_bea_1 = df_bea_1[~df_bea_1['Partei'].isnull()]
df_bea_1 = df_bea_1[df_bea_1['Name'] != 'Name/Nom']
df_bea_1 = df_bea_1[~df_bea_1['geschäftsname'].str.lower().str.contains('test')]

# Eine nicht plausible Abstimmung mit bloss ganz wenigen Teilnehmenden wird ausgefiltert. 
df_temp = df_bea_1.groupby(by=['datetime', 'geschäftsname', 'register', 'dateiname'])['stimme'].count().to_frame()
n = df_temp[df_temp['stimme'] < 10].reset_index()['geschäftsname'][0]
df_bea_1 = df_bea_1[df_bea_1['geschäftsname'] != n]

df_bea_1['anlage'] = 'bea1'
df_bea_1.to_csv('daten/daten_bea1.csv')
df_bea_1.head()

 50%|█████     | 11/22 [02:28<02:50, 15.46s/it]

In [ ]:
# Fehlende Felder (Wohnort) werden aus dem Grossrats-Datensatz ergänzt.  
df_bea_1 = pd.merge(df_bea_1, df_grossrat[['nachname', 'vorname', 'wohnort']], left_on=['Name', 'Vornamen'], right_on=['nachname', 'vorname'], how='left')
df_bea_1 = df_bea_1[['Name', 'Vornamen', 'wohnort', 'Partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'enthaltungen', 'keine_stimme', 'anlage', 'dateiname', 'register']]
df_bea_1.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
df_bea_1 = df_bea_1.drop_duplicates(subset=['nachname', 'vorname', 'partei', 'stimme', 'geschäftsname', 'dateiname', 'register'])

## BEA-Session 2

In [ ]:
# Die Excel-Dateien (Version 2) werden eingelesen. 
df_bea_2 = pd.DataFrame()

# Durch die Dateien loopen. 
for d in tqdm(format_bea_v2):
       
    # Durch die Register loopen.
    register = pd.ExcelFile('daten/bea2/' + d).sheet_names

    for r in register: 
        df_temp = pd.read_excel('daten/bea2/' + d, sheet_name=r)
        df_temp['dateiname'] = dateiname
        df_temp['register'] = r
        df_bea_2 = df_bea_2.append(df_temp)

# Platzhalter für fehlende Spalten werden eingefügt. 
df_bea_2['wohnort'] = ''
df_bea_2['anlage'] = 'bea2'

# Duplikate werden entfernt.
df_bea_2 = df_bea_2.drop_duplicates(subset=['Current Voting ID', 'Headline Text', 'Creation Date', 'Run Name', 'Handset ID', 'Name'])

# Die Resultate der einzelnen Abstimmungen werden berechnet.
df_res = df_bea_2.groupby(by=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])['Choice Text'].value_counts().to_frame().unstack()
df_res.columns = [x[1] for x in df_res.columns]
df_res.columns = ['res_abwesend', 'res_enthaltung', 'res_ja', 'res_nein']
df_res = df_res.reset_index()

# Die Angaben zu den Resultaten werden dem Datensatz hinzugefügt. 
df_bea_2 = pd.merge(df_bea_2, df_res, left_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'], right_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])

# Eine Datetime-Spalte wird hinzugefügt. 
for i, r in df_bea_2.iterrows():
    if str(r['Received Time']) == 'nan':
        df_bea_2.at[i, 'datetime'] = str(r['Creation Date'])
    else:
        df_bea_2.at[i, 'datetime'] = str(r['Creation Date'])[:-9] + ' ' + str(r['Received Time'])

# Um ganz sicher keine Abstimmungen zu verlieren, wird der "Run Name" dem "Headline Text" angehängt. 
df_bea_2['Headline Text'] = df_bea_2['Headline Text'] + '_' + df_bea_2['Run Name'].str.replace('Run ', '')

# Ein Datetime-Feld wird eingefügt. 
df_bea_2['datetime'] = [pd.to_datetime(x) for x in df_bea_2['datetime']]

# Duplikate werden entfernt. 
df_bea_2 = df_bea_2.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_2.head()

In [ ]:
# Fehlende Felder werden aus dem Grossrats-Datensatz ergänzt.  
df_grossrat['name-vorname'] = df_grossrat['nachname'] + ' ' + df_grossrat['vorname']
df_bea_2 = pd.merge(df_bea_2, df_grossrat[['nachname', 'vorname', 'wohnort', 'name-vorname']], left_on='Name', right_on='name-vorname', how='left')
df_bea_2 = df_bea_2.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_2 = df_bea_2[['nachname', 'vorname', 'wohnort_x', 'Vote Group', 'datetime', 'Headline Text', 'Choice Text', 'res_ja', 'res_nein', 'res_enthaltung', 'res_abwesend', 'anlage', 'dateiname', 'register']]
df_bea_2.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
del df_grossrat['name-vorname']

## Harmonisieren und zusammenfügen der Daten 

In [ ]:
# Zusammenfügen der Daten. 
df = df_rathaus.append(df_bea_1)
df = df.append(df_bea_2)

In [ ]:
# Einige Retouchen. 

# Einträge, die die Tabellenbeschriftungen enthalten, werden entfernt. 
df = df[df['partei'] != 'Partei']

# Die Datetime-Spalte wird ins Datetime-Format überführt. 
df['datetime'] = pd.to_datetime(df['datetime'], errors='raise')

# Die Angaben zur Stimme (ja, nein, Enthaltung etc. werden harmonisiert.)
df = df.dropna(subset=['stimme'])

def janein(stimme):
    if 'ja' in stimme.lower():
        return 'ja'
    elif 'nein' in stimme.lower():
        return 'nein'
    elif 'enthaltung' in stimme.lower():
        return 'enthaltung'
    elif 'abwesend' in stimme.lower() or '-' in stimme.lower():
        return 'abwesend'
    
df['stimme'] = df['stimme'].apply(janein)

In [ ]:
# Die Namen der Parteien sind sehr unterschiedlich erfasst. Sie werden nun harmonisiert. 
parteien_dict = {'SVP / UDC': 'SVP', 'SP-JUSO-PSA / PS-JS-PSA': 'SP', 'FDP / PLR': 'FDP',
                 'Grüne / Les Verts': 'Grüne', 'BDP / PBD': 'Die Mitte', 'glp / pvl': 'glp', 
                 'EVP / PEV': 'EVP', 'EDU / UDF': 'EDU', 'CVP / PDC': 'Die Mitte', 'BDP': 'Die Mitte',
                 'PLR': 'FDP', 'PSA': 'SP', 'CVP': 'Die Mitte', 'PS': 'SP', 'les Verts': 'Grüne',
                 'UDC': 'SVP', 'PEV': 'EVP', 'PDC': 'Die Mitte', 'SVP/UDC': 'SVP', 'FDP/PLR': 'FDP',
                 'Grüne/les Verts': 'Grüne', 'glp/plv': 'glp', 'EVP/PEV': 'EVP', 'EDU/UDF': 'EDU',
                 'Grünliberale': 'glp', 'Les Verts': 'Grüne'}

def harmos(partei):
    if partei in parteien_dict.keys(): 
        return parteien_dict[partei]
    else:
        return partei

df['partei'] = df['partei'].apply(harmos)

In [ ]:
'''# Einige Vornamen werden unterschiedlich geschrieben. Diese werden nun harmonisiert. 
vornamen = [['Ulrich', 'Frutiger', 'Ueli'], 
            #['Roulet Romy 	Sandra']]'''

In [ ]:
'''# Subset: ohne Wohnort. 
df_temp1 = df[~df['wohnort'].isnull()]
df_temp2 = df[df['wohnort'].isnull()]

# Merge
pd.merge(df_temp2, df_grossrat[['vorname', 'nachname', 'wohnort']], 
         left_on=['vorname', 'nachname'], right_on=['vorname', 'nachname'], how='left')
# Zusammenfügen'''

In [ ]:
# Die Daten werden exportiert. 
df.to_csv('daten/abstimmungen_grossrat.csv')